In [2]:
!pip install faiss-cpu

     ---------------------------------------- 10.8/10.8 MB 9.2 MB/s eta 0:00:00


In [4]:
!pip install datasets

     -------------------------------------- 474.6/474.6 kB 4.3 MB/s eta 0:00:00
     -------------------------------------- 134.3/134.3 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 21.5/21.5 MB 9.1 MB/s eta 0:00:00
     ------------------------------------- 319.8/319.8 kB 10.0 MB/s eta 0:00:00
     ------------------------------------- 236.8/236.8 kB 15.1 MB/s eta 0:00:00
     ---------------------------------------- 61.0/61.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:
      Successfully uninstalled huggingface-hub-0.10.1


In [44]:
!pip install --no-cache-dir transformers sentencepiece

In [49]:
!pip install transformers

In [50]:
!pip install sentencepiece

In [1]:
import pandas as pd
import numpy as np

import re
import torch
import faiss

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

### Посмотрим на датасет, источники новостей, темы статей и их количество

In [18]:
df = pd.read_csv('./russian_news.csv')
df

,source,title,text,date,tags
0,lenta.ru,Синий богатырь,В 1930-е годы Советский Союз охватила лихорадк...,2020-08-29 21:01:00+00:00,Все
1,lenta.ru,Загитова согласилась вести «Ледниковый период»,Олимпийская чемпионка по фигурному катанию Ал...,2020-08-31 17:04:00+00:00,Все
2,lenta.ru,Объяснена опасность однообразного питания,Российский врач-диетолог Римма Мойсенко объясн...,2020-08-31 17:07:00+00:00,Все
3,lenta.ru,«Предохраняться? А зачем?»,В 2019 году телеканал «Ю» запустил адаптацию з...,2020-08-29 21:04:00+00:00,Все
4,lenta.ru,Ефремов систематически употреблял наркотики,Актер Михаил Ефремов систематически употребл...,2020-08-31 15:27:00+00:00,Все
...,...,...,...,...,...
822643,lenta.ru,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,2019-12-14 00:00:00+00:00,ТВ и радио
822644,lenta.ru,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,2019-12-14 00:00:00+00:00,Все
822645,lenta.ru,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,2019-12-14 00:00:00+00:00,Политика
822646,lenta.ru,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,2019-12-14 00:00:00+00:00,Общество


In [19]:
df.source.value_counts()

lenta.ru       805485
ria.ru          14755
meduza.io        1538
tjournal.ru       870
Name: source, dtype: int64

In [20]:
df.tags.value_counts()[:50]

Все                     460683
Политика                 40716
Общество                 35236
Украина                  22523
Происшествия             19825
Госэкономика             15476
Футбол                   15239
Кино                     10720
Интернет                  9319
Бизнес                    9146
Следствие и суд           9133
Наука                     7932
Музыка                    7406
Люди                      6567
Квартира                  5180
Преступность              5057
ТВ и радио                4391
Космос                    4311
Конфликты                 3798
События                   3664
Coцсети                   3635
Бокс и ММА                3549
Регионы                   3516
Летние виды               3265
Деловой климат            3254
Явления                   3216
Криминал                  3209
Звери                     3063
Рынки                     2834
Мир                       2566
Стиль                     2566
Оружие                    2457
Город   

In [21]:
df.tags.nunique()

11495

In [22]:
df.text.describe()

count            822643
unique           821707
top       РИА "Новости"
freq                291
Name: text, dtype: object

### Уберем из датасета строки с повторяющимися статьями

In [23]:
dfr_drop = df.drop_duplicates(subset=['text'])
dfr_drop

,source,title,text,date,tags
0,lenta.ru,Синий богатырь,В 1930-е годы Советский Союз охватила лихорадк...,2020-08-29 21:01:00+00:00,Все
1,lenta.ru,Загитова согласилась вести «Ледниковый период»,Олимпийская чемпионка по фигурному катанию Ал...,2020-08-31 17:04:00+00:00,Все
2,lenta.ru,Объяснена опасность однообразного питания,Российский врач-диетолог Римма Мойсенко объясн...,2020-08-31 17:07:00+00:00,Все
3,lenta.ru,«Предохраняться? А зачем?»,В 2019 году телеканал «Ю» запустил адаптацию з...,2020-08-29 21:04:00+00:00,Все
4,lenta.ru,Ефремов систематически употреблял наркотики,Актер Михаил Ефремов систематически употребл...,2020-08-31 15:27:00+00:00,Все
...,...,...,...,...,...
822643,lenta.ru,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,2019-12-14 00:00:00+00:00,ТВ и радио
822644,lenta.ru,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,2019-12-14 00:00:00+00:00,Все
822645,lenta.ru,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,2019-12-14 00:00:00+00:00,Политика
822646,lenta.ru,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,2019-12-14 00:00:00+00:00,Общество


In [24]:
dfr_drop.tags.value_counts()[:50]

Все                     459752
Политика                 40716
Общество                 35236
Украина                  22523
Происшествия             19825
Госэкономика             15476
Футбол                   15238
Кино                     10720
Интернет                  9319
Бизнес                    9145
Следствие и суд           9133
Наука                     7932
Музыка                    7405
Люди                      6566
Квартира                  5180
Преступность              5057
ТВ и радио                4391
Космос                    4311
Конфликты                 3798
События                   3664
Coцсети                   3635
Бокс и ММА                3549
Регионы                   3516
Летние виды               3265
Деловой климат            3254
Явления                   3216
Криминал                  3209
Звери                     3063
Рынки                     2834
Мир                       2566
Стиль                     2566
Оружие                    2457
Город   

In [25]:
dfr_drop.tags.nunique()

11495

In [26]:
dfr_drop.text.describe()

count                                                821707
unique                                               821707
top       В 1930-е годы Советский Союз охватила лихорадк...
freq                                                      1
Name: text, dtype: object

In [27]:
dfr_drop.source.value_counts()

lenta.ru       804548
ria.ru          14755
meduza.io        1538
tjournal.ru       867
Name: source, dtype: int64

### Отберем 10 тем и в каждой оставим по 10 статей

In [28]:
df_flat = dfr_drop[dfr_drop['tags'] == 'Квартира'][:10]
df_flat

,source,title,text,date,tags
515003,lenta.ru,Число бюджетных новостроек в Москве выросло в ...,По состоянию на конец 2013 года объем предложе...,2014-01-13 00:00:00+00:00,Квартира
515165,lenta.ru,В Москве выросла стоимость аренды самых дороги...,В декабре 2013 года средняя ставка в 100 самых...,2014-01-14 00:00:00+00:00,Квартира
515313,lenta.ru,Возле самого высокого здания ЕС появится новый...,"Рядом с небоскребом The Shard (""Осколок"") в Ло...",2014-01-15 00:00:00+00:00,Квартира
515491,lenta.ru,Самые дорогие квартиры Москвы подорожали на треть,В декабре 2013 года общая стоимость 100 самых ...,2014-01-16 00:00:00+00:00,Квартира
515527,lenta.ru,На московский поселок иностранцев нашлись поку...,Катарский суверенный фонд Qatar Investment Aut...,2014-01-16 00:00:00+00:00,Квартира
515546,lenta.ru,Столичную квартиру оценили в 411 подмосковных,По состоянию на начало 2014 года стоимость сам...,2014-01-16 00:00:00+00:00,Квартира
515585,lenta.ru,СМИ сообщили о стократном росте платы за ЖКХ в...,Стоимость коммунальных услуг в городе Апатиты ...,2014-01-16 00:00:00+00:00,Квартира
515635,lenta.ru,Москва отменила просроченный на 6 лет стройпроект,Мэрия Москвы признала утратившим силу постанов...,2014-01-17 00:00:00+00:00,Квартира
515694,lenta.ru,На берегу Невы построят две «императорские» вы...,Компания «Росстройинвест» купила земельный уча...,2014-01-17 00:00:00+00:00,Квартира
515712,lenta.ru,В России предложили строить бюджетные квартиры...,Минстрой разработал нормативы строительства жи...,2014-01-17 00:00:00+00:00,Квартира


In [29]:
df_football = dfr_drop[dfr_drop['tags'] == 'Футбол'][:10]
df_football

,source,title,text,date,tags
472229,lenta.ru,"""Локомотив"" перехватил футболиста у ""Ливерпуля""","Новичком московского футбольного клуба ""Локомо...",2013-01-15 00:00:00+00:00,Футбол
472290,lenta.ru,Футболист сборной России отверг подозрения в н...,"Нападающий московского футбольного ""Динамо"" Ал...",2013-01-15 00:00:00+00:00,Футбол
472297,lenta.ru,"Бывший тренер ""Спартака"" возглавил ""Севилью""",Бывший главный тренер московского футбольного ...,2013-01-15 00:00:00+00:00,Футбол
472305,lenta.ru,Сборная России по футболу сыграет с Бразилией,Сборная России по футболу проведет товарищески...,2013-01-15 00:00:00+00:00,Футбол
472366,lenta.ru,Бывший футболист сборной Грузии покончил с соб...,Бывший футболист национальной сборной Грузии и...,2013-01-15 00:00:00+00:00,Футбол
472391,lenta.ru,Марадона решил завязать с тренерской работой,"Диего Марадона заявил, что он больше не хочет ...",2013-01-16 00:00:00+00:00,Футбол
472436,lenta.ru,"Забитый на последней минуте гол вывел ""Интер"" ...","Миланский ""Интер"" в дополнительное время сумел...",2013-01-16 00:00:00+00:00,Футбол
472438,lenta.ru,"""Реал"" обыграл ""Валенсию"" в 1/4 финала Кубка И...","Мадридский ""Реал"" во вторник, 15 января, в пер...",2013-01-16 00:00:00+00:00,Футбол
472439,lenta.ru,Самуэль Это'О создал комикс о самом себе,"Форвард махачкалинского футбольного клуба ""Анж...",2013-01-16 00:00:00+00:00,Футбол
472455,lenta.ru,Гвардиола заявил о желании поработать в чемпио...,"Бывший главный тренер испанской ""Барселоны"" Хо...",2013-01-16 00:00:00+00:00,Футбол


In [30]:
df_cinema = dfr_drop[dfr_drop['tags'] == 'Кино'][:10]
df_cinema

,source,title,text,date,tags
248564,lenta.ru,Уэсли Снайпса приговорили к трем годам тюрьмы,Американского актера Уэсли Снайпса в четверг п...,2008-04-25 00:00:00+00:00,Кино
472054,lenta.ru,"Режиссер последнего ""Бонда"" займется сериалом ...","Режиссер Сэм Мендес, снявший ""007: Координаты ...",2013-01-14 00:00:00+00:00,Кино
472056,lenta.ru,Жизнь Джона Макафи экранизируют,Кинокомпания Warner Bros. снимет фильм о жизни...,2013-01-14 00:00:00+00:00,Кино
472128,lenta.ru,СМИ сообщили об уходе Кайли Миноуг из музыки в...,Австралийская певица Кайли Миноуг решила покин...,2013-01-14 00:00:00+00:00,Кино
472131,lenta.ru,"""Мой мальчик"" по Нику Хорнби станет сериалом",Телеканал NBC заказал съемки пилотного эпизода...,2013-01-14 00:00:00+00:00,Кино
472166,lenta.ru,"Cоздатель ""Тюдоров"" сделает сериал про Алексан...","Английский сценарист Майкл Херст, известный по...",2013-01-14 00:00:00+00:00,Кино
472180,lenta.ru,"Главные героини ""Секса в большом городе"" встре...","Знакомство четырех центральных персонажей ""Сек...",2013-01-14 00:00:00+00:00,Кино
472198,lenta.ru,"""Пираты Карибского моря 5"" получили сценариста","Фильм ""Пираты Карибского моря 5"" будет снят по...",2013-01-14 00:00:00+00:00,Кино
472203,lenta.ru,Мозг Чарли Шина вскрыли в новом трейлере,"Вышел трейлер нового фильма Романа Копполы ""Ум...",2013-01-14 00:00:00+00:00,Кино
472211,lenta.ru,"Спилберг займется ""Парком Юрского периода 4""","Стивен Спилберг станет продюсером фильма ""Парк...",2013-01-14 00:00:00+00:00,Кино


In [31]:
df_internet = dfr_drop[dfr_drop['tags'] == 'Интернет'][:10]
df_internet

,source,title,text,date,tags
472060,lenta.ru,Главред Colta.ru рассказала о финансировании с...,Интернет-проект Colta.ru отказался от инвестиц...,2013-01-14 00:00:00+00:00,Интернет
472063,lenta.ru,МВД запустило мультимедийный сайт,"МВД России обновило официальный сайт, в которы...",2013-01-14 00:00:00+00:00,Интернет
472162,lenta.ru,"""Лаборатория Касперского"" раскрыла международн...","Антивирусная ""Лаборатория Касперского"" обнаруж...",2013-01-14 00:00:00+00:00,Интернет
472214,lenta.ru,На антарктической станции «Восток» появился ин...,У российских полярников на антарктической стан...,2013-01-15 00:00:00+00:00,Интернет
472343,lenta.ru,Запуск MySpace приурочили к выходу сингла Джас...,Сайт MySpace открылся для широкой публики. Об ...,2013-01-15 00:00:00+00:00,Интернет
472374,lenta.ru,Сайт вопросов и ответов Quora стал книгой,"Сайт вопросов и ответов Quora выпустил книгу, ...",2013-01-15 00:00:00+00:00,Интернет
472389,lenta.ru,Журналистам запретили брать фотографии из Twitter,Судья окружного суда Манхэттена Элисон Натан (...,2013-01-16 00:00:00+00:00,Интернет
472410,lenta.ru,Журналист CNET уволился из-за вмешательства в ...,Один из авторов интернет-издания CNET уволился...,2013-01-16 00:00:00+00:00,Интернет
472425,lenta.ru,Бывший и.о. главы Yahoo! займется изданием The...,Бывший исполняющий обязанности гендиректора Ya...,2013-01-16 00:00:00+00:00,Интернет
472445,lenta.ru,Facebook представил поиск по контенту,Социальная сеть Facebook представила поиск по ...,2013-01-16 00:00:00+00:00,Интернет


In [32]:
df_business = dfr_drop[dfr_drop['tags'] == 'Бизнес'][:10]
df_business

,source,title,text,date,tags
442996,lenta.ru,РЖД разрешит пассажирам путешествовать с автом...,"""Федеральная пассажирская компания"", входящая ...",2012-05-23 00:00:00+00:00,Бизнес
466367,lenta.ru,РЖД впервые перевезла пассажиров с личными авт...,Вечером 26 ноября 2012 года из Москвы в Хельси...,2012-11-27 00:00:00+00:00,Бизнес
472029,lenta.ru,Крупнейший производитель часов в мире купил ог...,"Швейцарская Swatch Group, крупнейший в мире пр...",2013-01-14 00:00:00+00:00,Бизнес
472050,lenta.ru,"""Дочка"" ""Росатома"" станет единоличным владельц...","Урановый холдинг ""Атомредметзолото"" (АРМЗ), до...",2013-01-14 00:00:00+00:00,Бизнес
472072,lenta.ru,Временный руководитель крупнейшего ритейлера Р...,Наблюдательный совет крупнейшего ритейлера Рос...,2013-01-14 00:00:00+00:00,Бизнес
472102,lenta.ru,Бывший глава Госстроя нацелился на долю в круп...,"Компания ""Нефтегазиндустрия"", принадлежащая бы...",2013-01-14 00:00:00+00:00,Бизнес
472160,lenta.ru,"Росавиация продлила обслуживание ""Аэросвита"" н...",Аэронавигационное обслуживание украинской авиа...,2013-01-14 00:00:00+00:00,Бизнес
472168,lenta.ru,"""Мечел"" закрыл сделку по покупке порта ""Ванино""","Металлургический холдинг ""Мечел"" закрыл сделку...",2013-01-14 00:00:00+00:00,Бизнес
472182,lenta.ru,Volkswagen впервые продал более девяти миллион...,Группа Volkswagen по итогам 2012 года увеличил...,2013-01-14 00:00:00+00:00,Бизнес
472185,lenta.ru,Количество легковых автомобилей в России вырос...,В России по итогам 2012 года было зарегистриро...,2013-01-14 00:00:00+00:00,Бизнес


In [33]:
df_science = dfr_drop[dfr_drop['tags'] == 'Наука'][:10]
df_science

,source,title,text,date,tags
472040,lenta.ru,Генетики научились восстанавливать цвет глаз п...,Польско-голландская группа генетиков научилась...,2013-01-14 00:00:00+00:00,Наука
472153,lenta.ru,Жидкий металл научили сохранять форму,"Австралийские инженеры создали нанопокрытие, к...",2013-01-14 00:00:00+00:00,Наука
472154,lenta.ru,Американские полярники прибыли к подледному ан...,Команда американских полярников прибыла к подл...,2013-01-14 00:00:00+00:00,Наука
472319,lenta.ru,США поможет России принимать данные с «Радиоас...,Россия заключила с США договор об использовани...,2013-01-15 00:00:00+00:00,Наука
472325,lenta.ru,ESA объявило сбор идей по защите от астероидов,Европейское космическое агентство (ESA) в рамк...,2013-01-15 00:00:00+00:00,Наука
472334,lenta.ru,Шимпанзе проявили склонность к справедливости,"Ученые установили, что шимпанзе склонны ""по-че...",2013-01-15 00:00:00+00:00,Наука
472364,lenta.ru,Генетики обнаружили связь австралийских абориг...,"Немецкие генетики установили, что некоторые гр...",2013-01-15 00:00:00+00:00,Наука
472423,lenta.ru,«Кьюриосити» выбрал необычную цель для первого...,Объектом первого бурения «Кьюриосити» выбраны ...,2013-01-16 00:00:00+00:00,Наука
472503,lenta.ru,Датирование средневековых хартий доверили комп...,Ученые из университета Торонто создали програм...,2013-01-16 00:00:00+00:00,Наука
472519,lenta.ru,В глобальном потеплении обвинили сажу,Международная группа ученых опубликовала докла...,2013-01-16 00:00:00+00:00,Наука


In [34]:
df_music = dfr_drop[dfr_drop['tags'] == 'Музыка'][:10]
df_music

,source,title,text,date,tags
471804,lenta.ru,Руководитель Берлинского филармонического орке...,Английский маэстро сэр Саймон Рэттл покинет по...,2013-01-11 00:00:00+00:00,Музыка
472064,lenta.ru,Джастин Тимберлейк анонсировал новый альбом,Актер и певец Джастин Тимберлейк анонсировал с...,2013-01-14 00:00:00+00:00,Музыка
472066,lenta.ru,Black Sabbath придумали название новому альбому,Новый альбом рок-группы Black Sabbath выйдет в...,2013-01-14 00:00:00+00:00,Музыка
472099,lenta.ru,Хедлайнером фестиваля KUBANA-2013 станет Syste...,"Одним из хедлайнеров фестиваля KUBANA, который...",2013-01-14 00:00:00+00:00,Музыка
472199,lenta.ru,Теодор Курентзис дебютирует в Венском филармон...,Венский филармонический оркестр впервые выступ...,2013-01-14 00:00:00+00:00,Музыка
472201,lenta.ru,"Бурановские бабушки"" превратятся в фестиваль","В Удмуртии состоится фестиваль ""Бурановские ба...",2013-01-14 00:00:00+00:00,Музыка
472202,lenta.ru,Названа причина отмены российского тура Мэрили...,Американский рок-музыкант Мэрилин Мэнсон отлож...,2013-01-14 00:00:00+00:00,Музыка
472204,lenta.ru,Вдова Джорджа Харрисона попросила не ставить е...,Кампания по установке памятника Джорджу Харрис...,2013-01-14 00:00:00+00:00,Музыка
472212,lenta.ru,Дэвид Боуи спел о современной истории России,Продюсер британского рок-музыканта Дэвида Боуи...,2013-01-14 00:00:00+00:00,Музыка
472303,lenta.ru,Yeah Yeah Yeahs анонсировали альбом,Американская группа Yeah Yeah Yeahs анонсирова...,2013-01-15 00:00:00+00:00,Музыка


In [35]:
df_people = dfr_drop[dfr_drop['tags'] == 'Люди'][:10]
df_people

,source,title,text,date,tags
471968,lenta.ru,Бритни Спирс расторгла помолвку и ушла из жюри...,Певица Бритни Спирс объявила о расторжении пом...,2013-01-12 00:00:00+00:00,Люди
472006,lenta.ru,"Титул ""Мисс Америка"" выиграла уроженка Бруклина",Уроженка нью-йоркского района Бруклин завоевал...,2013-01-13 00:00:00+00:00,Люди
472034,lenta.ru,Джоди Фостер рассказала о своей нетрадиционной...,Актриса Джоди Фостер впервые публично затронул...,2013-01-14 00:00:00+00:00,Люди
472061,lenta.ru,Киевляне вступились за акулу из торгового центра,В Киеве прошла акция протеста против администр...,2013-01-14 00:00:00+00:00,Люди
472069,lenta.ru,Оливия Уайлд собралась замуж,Актриса Оливия Уайлд обручилась со своим возлю...,2013-01-14 00:00:00+00:00,Люди
472209,lenta.ru,Принц Уильям и Кейт Миддлтон станут родителями...,Принц Уильям и его супруга Кэтрин Миддлтон ста...,2013-01-14 00:00:00+00:00,Люди
472365,lenta.ru,"Китайский детский сад ""поженил"" детей",В детском саду городского округа Чжэнчжоу в ки...,2013-01-15 00:00:00+00:00,Люди
472390,lenta.ru,В библиотеке Айовы запретят спать,"Публичная библиотека Айвоа-Сити, штат Айова, в...",2013-01-16 00:00:00+00:00,Люди
472406,lenta.ru,Чарли Шин станет дедом,Чарли Шин готовится впервые стать дедушкой. Об...,2013-01-16 00:00:00+00:00,Люди
472418,lenta.ru,Легендарного австралийского преступника похоро...,Знаменитый австралийский преступник Нед Келли ...,2013-01-16 00:00:00+00:00,Люди


In [36]:
df_crime = dfr_drop[dfr_drop['tags'] == 'Преступность'][:10]
df_crime

,source,title,text,date,tags
296991,lenta.ru,В убийстве американского радиоведущего признал...,В Нью-Йорке арестован подозреваемый в убийстве...,2009-03-25 00:00:00+00:00,Преступность
376049,lenta.ru,Выходец из Сомали планировал теракт в Портленде,В городе Портленд штата Орегон вечером 26 нояб...,2010-11-27 00:00:00+00:00,Преступность
466641,lenta.ru,Британские убийцы пожаловались в Страсбург на ...,"Три убийцы, отбывающие наказание в Великобрита...",2012-11-28 00:00:00+00:00,Преступность
472106,lenta.ru,В Лос-Анджелесе поймали захватчиков 14 заложников,Полиция Лос-Анджелеса арестовала двух подозрев...,2013-01-14 00:00:00+00:00,Преступность
472178,lenta.ru,В Индии произошло новое изнасилование в автобусе,Полиция задержала шестерых подозреваемых в нов...,2013-01-14 00:00:00+00:00,Преступность
472213,lenta.ru,Убийцей лидера калифорнийских неонацистов приз...,В США 12-летнего Джозефа Холла (Joseph Hall) п...,2013-01-15 00:00:00+00:00,Преступность
472248,lenta.ru,В Берлине банк ограбили через 30-метровый подкоп,В Берлине преступники ограбили банк Berliner V...,2013-01-15 00:00:00+00:00,Преступность
472354,lenta.ru,В Германии начался процесс над российскими шпи...,"В Штутгарте во вторник, 15 января, начался про...",2013-01-15 00:00:00+00:00,Преступность
472435,lenta.ru,В Сент-Луисе преступник открыл огонь в бизнес-...,"Преступник во вторник, 15 января, открыл огонь...",2013-01-16 00:00:00+00:00,Преступность
472514,lenta.ru,В Алжире исламисты похитили иностранцев с газо...,В Алжире исламисты захватили в заложники неско...,2013-01-16 00:00:00+00:00,Преступность


In [37]:
df_space = dfr_drop[dfr_drop['tags'] == 'Космос'][:10]
df_space

,source,title,text,date,tags
472169,lenta.ru,Роскосмос опубликовал планы до 2020 года,Роскосмос опубликовал основные положения прогр...,2013-01-14 00:00:00+00:00,Космос
472191,lenta.ru,NASA подписало договор на создание надувного м...,Американское космическое агентство подписало с...,2013-01-14 00:00:00+00:00,Космос
472215,lenta.ru,Аппарат «Луна-Глоб-1» запустят с космодрома «В...,Запуск космического аппарата «Луна-Глоб-1» буд...,2013-01-15 00:00:00+00:00,Космос
472637,lenta.ru,Астрономы засняли черное облако в Волке,"Европейские астрономы, управляющие расположенн...",2013-01-17 00:00:00+00:00,Космос
472694,lenta.ru,Запуск российского фотоспутника сорван из-за о...,Запуск российского космического аппарата зонди...,2013-01-17 00:00:00+00:00,Космос
472809,lenta.ru,Астрофизики недооценили скорость роста сверхма...,"Австралийские астрофизики обнаружили, что свер...",2013-01-18 00:00:00+00:00,Космос
472826,lenta.ru,NASA отправило «Мону Лизу» на Луну,Инженеры американского космического агентства ...,2013-01-18 00:00:00+00:00,Космос
472876,lenta.ru,«Кассини» обнаружил исчезновение кратеров на Т...,"Ученые, работающие с данными аппарата «Кассини...",2013-01-18 00:00:00+00:00,Космос
472942,lenta.ru,Вспышку радиоактивности «красного распятия» об...,Астрофизики из Йенского университета предложил...,2013-01-21 00:00:00+00:00,Космос
473068,lenta.ru,Астрономы доказали отсутствие крупных астероид...,"Астрономам NASA, работающим с данными аппарато...",2013-01-21 00:00:00+00:00,Космос


In [38]:
dfsList = []

In [40]:
dfsList.extend(value for name, value in locals().items() if name.startswith('df_'))

In [41]:
dfsList

[          source                                              title  \
 515003  lenta.ru  Число бюджетных новостроек в Москве выросло в ...   
 515165  lenta.ru  В Москве выросла стоимость аренды самых дороги...   
 515313  lenta.ru  Возле самого высокого здания ЕС появится новый...   
 515491  lenta.ru  Самые дорогие квартиры Москвы подорожали на треть   
 515527  lenta.ru  На московский поселок иностранцев нашлись поку...   
 515546  lenta.ru      Столичную квартиру оценили в 411 подмосковных   
 515585  lenta.ru  СМИ сообщили о стократном росте платы за ЖКХ в...   
 515635  lenta.ru  Москва отменила просроченный на 6 лет стройпроект   
 515694  lenta.ru  На берегу Невы построят две «императорские» вы...   
 515712  lenta.ru  В России предложили строить бюджетные квартиры...   
 
                                                      text  \
 515003  По состоянию на конец 2013 года объем предложе...   
 515165  В декабре 2013 года средняя ставка в 100 самых...   
 515313  Рядом с неб

In [42]:
# main_df = main_df[0:0] 
main_df = pd.concat(dfsList, ignore_index=True)

# нужно ли, чтобы нейросеть генеировала еще и название статьи?

main_df = main_df.drop(['source', 'date'], axis=1)
main_df

,title,text,tags
0,Число бюджетных новостроек в Москве выросло в ...,По состоянию на конец 2013 года объем предложе...,Квартира
1,В Москве выросла стоимость аренды самых дороги...,В декабре 2013 года средняя ставка в 100 самых...,Квартира
2,Возле самого высокого здания ЕС появится новый...,"Рядом с небоскребом The Shard (""Осколок"") в Ло...",Квартира
3,Самые дорогие квартиры Москвы подорожали на треть,В декабре 2013 года общая стоимость 100 самых ...,Квартира
4,На московский поселок иностранцев нашлись поку...,Катарский суверенный фонд Qatar Investment Aut...,Квартира
...,...,...,...
95,Астрофизики недооценили скорость роста сверхма...,"Австралийские астрофизики обнаружили, что свер...",Космос
96,NASA отправило «Мону Лизу» на Луну,Инженеры американского космического агентства ...,Космос
97,«Кассини» обнаружил исчезновение кратеров на Т...,"Ученые, работающие с данными аппарата «Кассини...",Космос
98,Вспышку радиоактивности «красного распятия» об...,Астрофизики из Йенского университета предложил...,Космос


In [43]:
main_df.tags.unique()

array(['Квартира', 'Футбол', 'Кино', 'Интернет', 'Бизнес', 'Наука',
       'Музыка', 'Люди', 'Преступность', 'Космос'], dtype=object)

In [44]:
# Задаем путь к предобученной модели

model_ckpt = "papluca/xlm-roberta-base-language-detection"

In [45]:
# Инициализируем токенизатор и модель, а также функции, которые возвращают эмбеддинги

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(end='')

Some weights of the model checkpoint at papluca/xlm-roberta-base-language-detection were not used when initializing XLMRobertaModel: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at papluca/xlm-roberta-base-language-detection and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [46]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]


def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, 
                              truncation=True, return_tensors='pt')
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    return cls_pooling(model_output)


# Посчитаем эмбединги в специальной структуре
# Определять 'близость' запроса и темы из датасета будем по эмбеддингам тэгов 

ds = Dataset.from_pandas(main_df)
embeddings_dataset = ds.map(
    lambda x: {'theme_embeddings': get_embeddings(x['tags']).detach().cpu().numpy()[0]})
embeddings_dataset.add_faiss_index(column='theme_embeddings')

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'text', 'tags', 'theme_embeddings'],
    num_rows: 100
})

In [47]:
# Создадим функцию, которая вернет 'близкие' 10 текстов

def get_sequences(question):
    question_embedding = get_embeddings([question]).cpu().detach().numpy()
    scores, samples = embeddings_dataset.get_nearest_examples(
        'theme_embeddings', question_embedding, k=10)
    return samples['text']

In [48]:
# Зададим вопрос и получим упорядоченный список от наиболее вероятных к наименее

question = 'Интернет'
base_articles = get_sequences(question)

In [49]:
base_articles

['Интернет-проект Colta.ru отказался от инвестиционной модели и превратился в общественное СМИ. Об этом "Ленте.ру" рассказала главный редактор "Кольты" Мария Степанова. "Наша ситуация - особенная, - рассказала Степанова. - Это значит, что у нас нету инвестора, нету собственника, сайт не принадлежит никому, кроме своей редакции и своих читателей, и финансируется исключительно обществом". На сегодняшний день редакции удалось собрать около 75 процентов годового бюджета сайта. Эти деньги, по словам Степановой, "Кольте" предоставили люди, которые войдут в попечительский совет сайта. Численность и состав совета, а также его функции, будут названы позже (на сайте 14 января появилась информация, что в состав совета может войти любой человек, членство обойдется в 300 тысяч рублей в год). Оставшиеся деньги - 2 миллиона 700 тысяч рублей - редакция намеревается искать самостоятельно. Первый транш, 400 тысяч рублей, "Кольта" надеется собрать с помощью краудфандинга на сайте "Планета.ру" - летом 201

In [51]:
type(base_articles[0])

str

In [70]:
from transformers import GPT2Tokenizer, GPT2Model

In [97]:
model = GPT2Model.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Using pad_token, but it is not set yet.


In [112]:
input_ids = tokenizer(
    [WHITESPACE_HANDLER(el) for el in base_articles],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=1024
)["input_ids"]

# ds_gen = Dataset.from_pandas(pd.DataFrame({'id': range(len(base_articles)), 'text': base_articles}))
# embeddings_dataset = ds_gen.map(
#     lambda x: {'text_embeddings': get_embeddings(x['text']).detach().cpu().numpy()[0]})
embeddings_dataset.add_faiss_index(column='text_embeddings')

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'text', 'text_embeddings'],
    num_rows: 10
})

In [115]:
output = model(**input_ids)
tokenizer.batch_decode(output, skip_special_tokens=True)

TypeError: GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
) argument after ** must be a mapping, not Tensor

In [109]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [110]:
print(tokenizer.decode(output))

ValueError: invalid literal for int() with base 10: 'last_hidden_state'

In [ ]:
prompt = "The Permaculture Design Principles are a set of universal design principles \
that can be applied to any location, climate and culture, and they allow us to design \
the most efficient and sustainable human habitation and food production systems. \
Permaculture is a design system that encompasses a wide variety of disciplines, such \
as ecology, landscape design, environmental science and energy conservation, and the \
Permaculture design principles are drawn from these various disciplines. Each individual \
design principle itself embodies a complete conceptual framework based on sound \
scientific principles. When we bring all these separate  principles together, we can \
create a design system that both looks at whole systems, the parts that these systems \
consist of, and how those parts interact with each other to create a complex, dynamic, \
living system. Each design principle serves as a tool that allows us to integrate all \
the separate parts of a design, referred to as elements, into a functional, synergistic, \
whole system, where the elements harmoniously interact and work together in the most \
efficient way possible."

In [69]:
checkpoint = "google/pegasus-xsum"
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


input_ids = tokenizer(
    [WHITESPACE_HANDLER(el) for el in base_articles],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=1024
)["input_ids"]


outputs = model.generate(**inputs, num_beams=5, num_beam_groups=5, max_new_tokens=500)
tokenizer.decode(outputs[0], skip_special_tokens=True)

"The BBC's Paul Adams looks at the key stories of the week, including the fallout from the Grenfell Tower fire, the UK's future in Europe and the fallout from the Brexit vote."

In [63]:
# 'XLMRobertaForCausalLM'

In [64]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", use_fast=False)

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

In [62]:

article_text = '''
Интернет-проект Colta.ru отказался от инвестиционной модели и превратился в общественное СМИ. Об этом "Ленте.ру" рассказала \
главный редактор "Кольты" Мария Степанова. "Наша ситуация - особенная, - рассказала Степанова. - Это значит, что у нас нету \
инвестора, нету собственника, сайт не принадлежит никому, кроме своей редакции и своих читателей, и финансируется\ 
исключительно обществом". На сегодняшний день редакции удалось собрать около 75 процентов годового бюджета сайта. Эти деньги,\
по словам Степановой, "Кольте" предоставили люди, которые войдут в попечительский совет сайта. Численность и состав совета,\
а также его функции, будут названы позже (на сайте 14 января появилась информация, что в состав совета может войти любой\
человек, членство обойдется в 300 тысяч рублей в год). Оставшиеся деньги - 2 миллиона 700 тысяч рублей - редакция\
намеревается искать самостоятельно. Первый транш, 400 тысяч рублей, "Кольта" надеется собрать с помощью краудфандинга на\
сайте "Планета.ру" - летом 2012 года с помощью этой платформы Colta.ru собрала 700 тысяч рублей, частично обеспечивших первые\
месяцы работы сайта вплоть до закрытия. На новой странице на "Планете" предусмотрены лоты от 100 рублей до искомых 2,7\
миллиона. По словам Степановой, столько же понадобится "Кольте" и на будущий год: 70-75 процентов от годового бюджета редакция\
вновь получит от попечителей. Сайт Colta.ru, запущенный командой старого OpenSpace в июле 2012 года и закрывшийся в ноябре\
из-за отсутствия инвесторов, возобновил работу 14 января 2013 года. Как рассказала Мария Степанова, первое время дизайн\
"Кольты" останется неизменным, хотя редакция планирует увеличить объем ежедневных обновлений по основным темам. Кроме того,\
14 января было объявлено, что Colta.ru станет партнером издания Slon.ru - некоторые материалы "Кольты" будут доступны на сайте\
"Слона". В марте 2013 года Colta.ru планирует провести перезапуск - в новом дизайне сайт станет больше похожа на старый\
OpenSpace, в частности вернутся тематические рубрики, которые в текущей версии "Кольты" скрыты. Кроме того, обновленный сайт\
предложит читателям подписку и другие удобные варианты оказания финансовой помощи "Кольте".
'''

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=1024
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=500,
    no_repeat_ngram_size=3,
    num_beams=4,
    max_new_tokens=500
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

Both `max_new_tokens` (=500) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Российский интернет-проект Colta.ru, закрывшийся в ноябре из-за отсутствия инвесторов, вновь возобновил работу.
